In [1]:
import pandas as pd  
import numpy as np 
import seaborn as sns

### Read in the data and check it's format for useability
- check the variable types 
- check for any missing data

In [4]:
shoe_orders_df = pd.read_csv('q1_data_set.csv')
shoe_orders_df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [5]:
shoe_orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


### Clean the data
- Before looking into anything, the data should be cleaned to avoid having to do this later on 
- Payment methods can be label/One Hot encoded (choose one hot encoding since payment methods are not ordinal data)
- Extract information from the created_at to have individual numerical features (easy to use later on)

In [6]:
shoe_orders_df['payment_method'].unique()

array(['cash', 'credit_card', 'debit'], dtype=object)

In [9]:
type(shoe_orders_df['created_at'][0])

str

In [ ]:
### Confirm some of the problem assumptions
- It is stated that there are 100 different shoe shops: check how many different shoe shows are in this data
- Also, look at total items bought per shop and graph the distribution
- This analysis is on a per shop basis